In [35]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
device = torch.device("cuda")

In [2]:
import sys
sys.path.append('./huggingface_models/')
sys.path.append('./utils/')
from sample_utils import *
from inference_utils import *
from codenet_process_utils import *
from self_training_utils import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
is_plbart = True
merged_filtered_dict = get_prepro_filtered_dict(None, is_plbart)
programs_dict = get_codenet_programs_dict(merged_filtered_dict)

In [5]:
plbart_sample_path = cached_path + 'plbart_codenet_preds_lang_dict.pkl'
with open(plbart_sample_path, 'rb') as infile:
     preds_lang_dict = pickle.load(infile)

In [60]:
plbart_sample_path = cached_path + 'codet5_codenet_preds_lang_dict.pkl'
with open(plbart_sample_path, 'rb') as infile:
     preds_lang_dict = pickle.load(infile)

In [65]:
plbart_sample_path = cached_path + 'codet5_full_codenet_preds_lang_dict.pkl'
with open(plbart_sample_path, 'rb') as infile:
     codet5_full_preds_lang_dict = pickle.load(infile)

In [67]:
codet5_full_preds_lang_dict.keys()

dict_keys([('C++', 'C#'), ('C#', 'Java'), ('C#', 'Python'), ('C#', 'C++'), ('Java', 'C#'), ('Java', 'Python'), ('Java', 'C++'), ('C', 'Java'), ('C', 'Python'), ('C', 'C#'), ('C', 'C++'), ('Java', 'C'), ('Python', 'C'), ('C#', 'C')])

In [71]:
with open(cached_path + "plbart_full_codenet_lang_pair_call_dict.pkl", 'rb') as infile:
    hypo_call_dict = pickle.load(infile)

In [72]:
hypo_call_dict.keys()

dict_keys([('C++', 'Java'), ('C++', 'C#'), ('C++', 'Python'), ('Java', 'C++'), ('Java', 'C#'), ('Java', 'Python'), ('C#', 'C++'), ('C#', 'Java'), ('C#', 'Python')])

In [69]:
with open(cached_path + "codet5_full_codenet_lang_pair_call_dict.pkl", 'rb') as infile:
    hypo_call_dict = pickle.load(infile)

In [7]:
call_dict_list = []
for i in range(41):
    call_dict_path = cached_path + 'codenet/codenet_call_dict_' + str(i) + '.json'
    with open(call_dict_path) as infile:
        call_dict = json.load(infile)
    call_dict_list.append(call_dict)


In [ ]:
# [programs, processed_results, result_keys, error_type_dict] = call_dict

In [12]:
lang1 = "Python"
lang2 = "Java"
# new_preds is preds after dedup; functions are new_preds after type filtering
# programs are functions with import_str
# 理论上只能用type filtering之后的(之前是不是也可以？因为之前也可能有Bug, 只不过通过type给过滤掉了)
# 用之后的，好处在于可以帮忙找到type无法过滤的bug
[new_preds, functions, function_id_dict, call_list] = hypo_call_dict[(lang1, lang2)]
[programs, processed_results, result_keys, error_type_dict] = call_list

In [33]:
sum_num = 0
for preds in new_preds:
    sum_num += len(preds)
print(sum_num)

226539


In [34]:
len(functions)

185787

In [57]:
hypo_pos_neg_dict = {}
for k, v in hypo_call_dict.items():
    [new_preds, functions, function_id_dict, call_list] = hypo_call_dict[k]
    [programs, processed_results, result_keys, error_type_dict] = call_list
    hypo_pos_neg_dict[k] = {}
    count0 = 0
    count1 = 0
    count2 = 0
    count3 = 0
    for i in range(len(new_preds)):
        function_ids = function_id_dict[i]
        if len(function_ids) ==0:
            count0 += 1
            continue
        if len(function_ids) ==1:
            count1 += 1
            continue
        results = [result_keys[x] for x in function_ids]
        results_set = set(results)
        if 'empty' in results_set:
            if len(results_set) > 1:
                hypo_pos_neg_dict[k][i] = {'pos':[], 'neg':[]}
                for j in function_ids:
                    function = functions[j]
                    result = result_keys[j]
                    if result == 'empty':
                        hypo_pos_neg_dict[k][i]['pos'].append(j)
                    else:
                        hypo_pos_neg_dict[k][i]['neg'].append(j)
            else:
                count3 += 1
        else:
            count2 += 1
    print(k, len(new_preds), len(hypo_pos_neg_dict[k]), count0, count1, count2, count3)
#     print(k, len(hypo_pos_neg_dict[k]))

('Java', 'C') 10511 794 5043 862 3470 342
('Python', 'C') 26911 3936 4417 586 17229 743
('C#', 'C') 3101 105 1923 327 672 74
('C++', 'C') 37126 5332 4565 4540 16771 5918
('C', 'Java') 12424 2727 1705 1164 4244 2584
('C', 'Python') 12424 5641 0 250 4436 2097
('C', 'C#') 12424 2485 1607 1586 4296 2450
('C', 'C++') 12424 3305 548 2249 2821 3501


In [58]:
# merge by lang
merged_hypo_pos_neg_dict = {lang:[] for lang in new_langs}
for lang1 in new_langs:
    for lang2 in new_langs:
        if lang1 == lang2:
            continue
        if (lang1, lang2) in hypo_pos_neg_dict:
            [new_preds, functions, function_id_dict, call_list] = hypo_call_dict[(lang1, lang2)]
            idx_list = list(hypo_pos_neg_dict[(lang1, lang2)].keys())
            idx_sorted = sorted(idx_list)
            for i in idx_sorted:
                pn_funtion_dict = {}
                pn_dict = hypo_pos_neg_dict[(lang1, lang2)][i]
                for key in pn_dict.keys():
                    pn_funtion_dict[key] = [functions[x] for x in pn_dict[key]]
                merged_hypo_pos_neg_dict[lang2].append(pn_funtion_dict)
        

In [59]:
for lang in new_langs:
    print(lang, len(merged_hypo_pos_neg_dict[lang]))

C++ 3305
Java 2727
Python 5641
C# 2485
C 10167


In [42]:
hypo_pos_neg_dict[21754]

{'pos': [148762, 148763], 'neg': [148759, 148760, 148761, 148764]}

In [28]:
len(result_keys)

185787

In [20]:
print(functions[0])

static void printStack() { for (int i = 1; i < 10; i++) { for (int j = 1; j < 10; j++) { System.out.print(i + " x" + " " + j + " = " + (i * j)); } } }
